In [ ]:
import os
from dotenv import load_dotenv
os.environ['DISABLE_MODEL_SOURCE_CHECK'] = 'True'

from paddleocr  import PaddleOCR, PaddleOCRVL, PPStructureV3
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2
import re
import json
import requests
import pprint


In [ ]:
# --- VERSI AKHIR: LENGKAP, BENAR, & TANPA ERROR ---

ocr_model = PaddleOCR(
    # --- PARAMETER UTAMA BAHASA ---
    lang='id',  # Otomatis memilih model Bahasa Indonesia

    # --- PARAMETER ORIENTASI & PELURUSAN ---
    use_doc_orientation_classify=True,  # Untuk rotasi gambar 90/180/270 derajat
    use_doc_unwarping=True,             # Untuk meluruskan struk yang kusut/melengkung
    doc_unwarping_model_name='UVDoc',
    use_textline_orientation=True,      # Untuk orientasi per baris teks
    
    # --- PARAMETER FINE-TUNING DETEKSI ---
    text_det_limit_side_len=1200,       
    text_det_limit_type='max',
    text_det_thresh=0.3,                
    text_det_box_thresh=0.6,            
    text_det_unclip_ratio=1.5,          
    
    # --- PARAMETER OUTPUT ---
    text_rec_score_thresh=0.5,          
    return_word_box=False,              
)

In [ ]:
img_path = os.path.join('..', 'data', 'struk', '6.jpeg')

In [ ]:
hasil = ocr_model.predict(
    img_path, 

    # 2. Unclip Ratio: KEMBALIKAN KE STANDARD (1.5 atau 1.6)
    # 2.0 terlalu besar untuk struk thermal yang rapat.
    text_det_unclip_ratio=1.5,

    # 3. Box Threshold: Naikkan sedikit (0.5 atau 0.6)
    # 0.4 terlalu sensitif, menangkap bayangan kertas sebagai kotak teks sampah.
    # 0.6 membuat kotak lebih selektif dan rapi.
    text_det_box_thresh=0.6,

    # 4. Det Threshold: Default (0.3)
    text_det_thresh=0.3,
    
    # Parameter Wajib Struk
    use_textline_orientation=True,
    use_doc_orientation_classify=True,
    text_rec_score_thresh=0.0,          # Tetap 0.0 agar semua teks masuk
)

In [ ]:
data_dict = hasil[0]

list_teks = data_dict.get('rec_texts', [])
list_skor = data_dict.get('rec_scores', [])
list_box  = data_dict.get('dt_polys', [])

In [ ]:
list_teks

In [ ]:
# Cari dan muat file .env dari direktori yang sama
load_dotenv()

# --- VERIFIKASI: Coba ambil API Key untuk memastikan berhasil ---
api_key = os.getenv('OPENROUTER_API_KEY')

if api_key:
    print("✅ Berhasil! API Key ditemukan.")
    # Hanya menampilkan 10 karakter pertama untuk keamanan
    print(f"API Key dimulai dengan: {api_key[:10]}...")
else:
    print("Gagal! API Key tidak ditemukan. Periksa kembali file .env Anda.")

In [ ]:
# 2. Definisikan Prompt (Bisa disesuaikan di masa depan)
PROMPT_NEUTRAL = """
Anda adalah ahli dalam mengekstraksi informasi dari berbagai macam struk belanja di Indonesia.
Dari daftar teks mentah hasil OCR berikut, ekstraklah informasi kunci dan formatlah hasilnya dalam bentuk JSON yang valid.

Teks Struk:
{list_teks}

Instruksi Umum:
- 'nama_toko': Identifikasi nama toko atau pedagang. Koreksi typo OCR yang umum jika nama toko sudah jelas.
- 'alamat': Gabungkan baris-baris yang relevan untuk membentuk alamat lengkap.
- 'tanggal': Ambil tanggal transaksi dan format sebagai YYYY-MM-DD.
- 'daftar_item': Identifikasi daftar barang yang dibeli. Setiap item adalah object dengan 'nama_barang', 'jumlah', dan 'harga'.
- 'subtotal', 'diskon', 'total_belanja', 'tunai', 'kembali': Ekstrak nilai numeriknya.
- 'metode Pembayaran': identifikasi metode pembayaran yang digunakan dalam proses belanja yan ada di struk belanja 

PENTING: Output Anda harus HANYA berupa JSON, tanpa teks tambahan, tanpa penjelasan, dan tanpa pembungkus markdown (```json ... ```).
"""

In [ ]:
# --- FUNSI UTAMA YANG UNIVERSAL ---

def parse_struk_from_image(image_path, ocr_model_instance):
    """
    Fungsi untuk memproses gambar struk dari path file menjadi data terstruktur.
    
    Args:
        image_path (str): Path ke file gambar struk.
        ocr_model_instance: Instance dari PaddleOCR yang sudah diinisialisasi.
        
    Returns:
        dict: Dictionary berisi data struk yang sudah diparsing, atau None jika gagal.
    """
    print(f"🚀 Memproses gambar: {image_path}")
    
    try:
        # --- LANGKAH 1: OCR ---
        print("1. Melakukan OCR...")
        hasil_ocr = ocr_model_instance.predict(image_path)
        list_teks = hasil_ocr[0].get('rec_texts', [])
        
        if not list_teks:
            print("OCR tidak menemukan teks apa pun.")
            return None
            
        print(f"OCR berhasil, menemukan {len(list_teks)} baris teks.")

        # --- LANGKAH 2: PARSING DENGAN LLM ---
        print("2. Mengirim ke LLM untuk parsing...")
        
        # Format prompt dengan list_teks
        prompt_final = PROMPT_NEUTRAL.format(list_teks=list_teks)
        
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json",
                "HTTP-Referer": "http://localhost:8888",
                "X-Title": "OCR Struk Notebook"
            },
            data=json.dumps({
                "model": "mistralai/mistral-7b-instruct:free",
                "messages": [
                    {"role": "system", "content": "Anda adalah asisten AI yang hanya merespons dengan format JSON yang valid."},
                    {"role": "user", "content": prompt_final}
                ]
            })
        )
        
        response.raise_for_status()
        response_data = response.json()
        llm_response_string = response_data['choices'][0]['message']['content']
        
        # --- LANGKAH 3: BERSIHKAN DAN PARSE JSON ---
        print("3. Membersihkan dan mem-parse hasil...")
        
        cleaned_string = llm_response_string.strip()
        if cleaned_string.startswith('```json'):
            cleaned_string = cleaned_string[7:]
        if cleaned_string.endswith('```'):
            cleaned_string = cleaned_string[:-3]
        cleaned_string = cleaned_string.strip()
        
        data_struk = json.loads(cleaned_string)
        
        print("   ✅ Proses selesai!")
        return data_struk

    except requests.exceptions.HTTPError as http_err:
        print(f"Error HTTP dari LLM: {http_err}")
        print(f"Respons Server: {response.text}")
    except json.JSONDecodeError as json_err:
        print(f"Gagal mem-parsing JSON: {json_err}")
        print("Respons Mentah LLM:", llm_response_string)
    except Exception as e:
        print(f"Terjadi error tidak terduga: {e}")
        
    return None

In [ ]:
# --- CONTOH PENGGUNAAN ---

# Pastikan ocr_model sudah ada. Jika tidak, jalankan ulang cell inisialisasinya.
# ocr_model = PaddleOCR(...) 

# Ganti dengan path ke gambar struk lain yang ingin Anda proses
path_ke_struk_baru = img_path

# Panggil fungsi dan simpan hasilnya
hasil_struk_baru = parse_struk_from_image(path_ke_struk_baru, ocr_model)

# Tampilkan hasilnya jika berhasil
if hasil_struk_baru:
    print("\n" + "="*50)
    print("     HASIL AKHIR PARSING STRUK")
    print("="*50)
    pprint.pprint(hasil_struk_baru)